In [ ]:
from llm.LLMModel import MistralAI, MistralAIInstruct
from ner.utils import run_command

checkpoint = "/myhome/Master-thesis/llm/mistralai/Mistral-7B-Instruct-v0.2ketl_training/130-2024-02-02_14:56:14_mistral-instruct_ft-data/"
merge_model_path = f"{checkpoint}to_trash/"
base_model_name = MistralAIInstruct(without_model = True).base_model_id
base_model_path = "/myhome/Master-thesis/llm/mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
command = f"python /myhome/Master-thesis/merge_lora_model.py {checkpoint} {merge_model_path}"
run_command(command)

In [ ]:
command = f"python llama.cpp/convert.py {merge_model_path} --outtype f16 --outfile {checkpoint}/merged.bin --vocab-dir {base_model_path} --vocab-type hfft"
run_command(command)

In [ ]:
quantization = 'q8_0'
command = f"llama.cpp/build/bin/quantize {checkpoint}/merged.bin {checkpoint}/merged_ggml_{quantization}.bin {quantization}"

run_command(command)